* CELEX: Nimm die SAMPA ground truth, konvertiere nach IPA mit dem selbstgebauten dict basierend auf der WIKI Tabelle. Dann nimm das IPA -> Buchstaben dict um die alternativen Schreibweisen zu generieren und gehe die manuell durch und sortiere aus
* ChildLex: Nimm alle Wörter aus dem Korpus und schaue nach welche Wörter mit SAMPA im CELEX Korpus existieren. Diejenigen die es gibt: Easy, da SAMPA existiert und sogar schon in IPA umgewandelt und sogar alternative writings da. Diejenigen die es nicht gibt (vermutlich wenige): Nimm den Online Konverter um Text in SAMPA umzuwandeln (das ist die ground truth! Achte darauf, dass Alphabet so wie beim CELEX Korpus). Wandel diese Wörter dann in IPA um und generiere alternative Schreibweisen
* Fibelwörter: Gleicher Ansatz wie bei ChildLex

In [2]:
# Task: Come up with alternative spellings for all orthographic words in a database
# Tool 1: IPA -> Graphem Konverter Wiki
# Tool 2: SAMPA -> IPA Konverter Wiki

# Idea: Take the phonetic sequence (SAMPA) of every word, convert it to an IPA sequence and
        # then convert that to all grapheme sequences
    
# Needs: A dictionary for SAMPA -> IPA (ideally non-ambiguous). A dictionary for IPA -> Text (ambigu.)

import tensorflow as tf
import numpy as np
import itertools
from sklearn.model_selection import train_test_split
import os

/Users/jannisborn/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
path = '/Users/jannisborn/Desktop/LDS_Data/celex2/german/gpl/gpl.cd'

def extract_celex(path):
    """
    Reads in data from the CELEX corpus
    
    Parameters:
    -----------
    PATH        {str} the path to the desired celex file, i.e. gpl.cd 
                    (contains orthography and phonology)

    Returns:
    -----------
    2 Tuples, each with 2 variables. 
        First tuple:
    W           {np.array} of words (length 51728) for gpl.cd
    P           {np.array} of phoneme sequences (length 51728) for gpl.cd
        Second tuple:
    WORD_DICT   {dict} allowing to map the numerical array W back to strings
    PHON_DICT   {dict} doing the same for the phonetical arrays P

    
    Call via:
    path = "/Users/jannisborn/Desktop/LDS_Data/celex2/german/gpl/gpl.cd"
    ((w,p) , (word_dict, phon_dict)) = extract_celex(path)
    
    """
    
    
    with open(path, 'r') as file:

        raw_data = file.read().splitlines()
        words = []
        phons = []
        m = 0
        t = 0
        for ind,raw_line in enumerate(raw_data):
            
            line = raw_line.split("\\")

            if line[-2]: # Use only words that HAVE a SAMPA transcript (reduces from 51k to 37345)

            # exclude foreign words that have the 'æ' tone (SAMPA '{' ) like in PoINte   - 18 words
            # exclude foreign words that have the 'ɑ' tone (SAMPA 'A' ) like in NuANce   - 28 words
            # exclude foreign words that have a nasal vowel (SAMPA '~' ) like in Jargon  - 22 words
                if not 'A' in line[-2] and not '{' in line[-2] and not '~' in line[-2]: 

                    if not ('tS' in line[-2] and not 'tsch' in line[1]): # exclude 9 foreign words like 'Image', 'Match', 'Punch', 'Sketch'

                        if len(line[1]) < 15 and len(line[-2]) < 15 : # exclude extra long words (reduces to 34376)
                            
                            if len(line[-2]) > m:
                                m = len(line[-2])
                                print(line[1],line[-2])
                                

                            words.append(line[1].lower()) # All words are lowercase only
                            phons.append(line[-2]) # Using SAMPA notation
                            
                        else:
                            t+=1
                            
    print("Excluded",t, "words because they were too long (more than 15 phons)" )
    print("Size of dataset is", len(words), "samples")

    return words,phons





def str_to_num_dataset(X,Y,pads=5):
    """
    This method receives 2 lists of strings (input X and output Y) and converts it to padded, numerical arrays.
    It returns the numerical dataset as well as the dictionaries to retrieve the strings.
    
    PADS    {int} specifiying how many additional fields should be padded (to allow long words to have longer alt. writings)
    """

    # 1. Define dictionaries 
    # Dictionary assignining a unique integer to each input character
    try:
        u_characters = set(' '.join(X)) 
    except TypeError:
        # Exception for TIMIT dataset (one phoneme is repr. by seq. of chars)
        print("TypeError occurred.")
        u_characters = set([quant for seq in X for quant in seq])

    char2numX = dict(zip(u_characters, range(1,len(u_characters)+1)))

    # Dictionary assignining a unique integer to each phoneme
    try:
        v_characters = set(' '.join(Y)) 
    except TypeError:
        print("TypeError occurred.")
        v_characters = set([quant for seq in Y for quant in seq])
    char2numY = dict(zip(v_characters, range(1,len(v_characters)+1))) # Using 0 causes trouble for tf.edit_distance
    
    # 2. Padding
    # Pad inputs
    char2numX['<GO>'] = len(char2numX) + 1
    char2numX['<PAD>'] = len(char2numX) + 1
    mx_l_X = max([len(word) for word in X]) # longest input sequence
    # Padd all X for the final form for the LSTM
    x = [[char2numX['<GO>']] + [char2numX['<PAD>']]*(mx_l_X - len(word)) +[char2numX[char] for char in word] for word in X]
    x = np.array(x) 

    # Pad targets
    char2numY['<GO>'] = len(char2numY) + 1 # Define number denoting the response onset
    char2numY['<PAD>'] = len(char2numY) + 1 
    mx_l_Y = max([len(phon_seq) for phon_seq in Y]) # longest output sequence

    y = [[char2numY['<GO>']] + pads*[char2numY['<PAD>']] + [char2numY['<PAD>']]*(mx_l_Y - len(ph_sq)) + [char2numY[phon] for phon in ph_sq] for ph_sq in Y]
    y = np.array(y)

    return ((x,y) , (char2numX,char2numY))


Save the dataset

In [7]:
words,phons = extract_celex(path)
((phons_num, words_num), (phon_dict, word_dict)) = str_to_num_dataset(phons,words)
    
print(len(word_dict),word_dict)
print(len(phon_dict),phon_dict)
print(words_num.shape, phons_num.shape)
print(words_num[321,:], phons_num[321,:])

np.savez("data/celex.npz", words=words_num, phons=phons_num, word_dict=word_dict, phon_dict=phon_dict)

A a:
aalen a:l
aalglatt a:l#glat
Aasgeier a:z#gai@r
abaenderlich ap#End@r#lIx
abbauwuerdig ap#bau#vYrdIx
Abbrucharbeit ap#brEx#arbait
Excluded 5593 words because they were too long (more than 15 phons)
Size of dataset is 31675 samples
29 {'y': 1, 's': 2, 'e': 3, 'w': 4, 'i': 5, 'k': 6, ' ': 7, 'm': 8, 'r': 9, 'q': 10, 'c': 11, 'p': 12, 'o': 13, 'u': 14, 'h': 15, 'd': 16, 'a': 17, 'n': 18, 'g': 19, 'z': 20, 'f': 21, 't': 22, 'b': 23, 'v': 24, 'l': 25, 'x': 26, 'j': 27, '<GO>': 28, '<PAD>': 29}
40 {'y': 1, 's': 2, 'e': 3, '#': 4, '@': 5, 'i': 6, 'k': 7, ' ': 8, 'm': 9, 'U': 10, 'I': 11, 'r': 12, 'p': 13, 'S': 14, 'u': 15, 'o': 16, 'h': 17, '|': 18, 'd': 19, ':': 20, '/': 21, 'a': 22, 'E': 23, 'n': 24, 'g': 25, 'O': 26, 'z': 27, 'f': 28, 'Z': 29, 't': 30, 'b': 31, 'N': 32, 'v': 33, 'l': 34, 'Y': 35, 'x': 36, 'j': 37, '+': 38, '<GO>': 39, '<PAD>': 40}
(31675, 20) (31675, 15)
[28 29 29 29 29 29 29 29 29 29 29 29 29 29 17 23 25 14 21 22] [39 40 40 40 40 40 40 40 22 13  4 34 10 28 30]


In [8]:
data = np.load('data/celex.npz')
sampa_dict = {key:data['phon_dict'].item().get(key) for key in data['phon_dict'].item()}
sampa_keys = list(sampa_dict.keys()) # Has 43 keys originally, 40 after excluding {, ~, A
print("Length of phonetic dict is ", len(sampa_dict), " and the keys are: \n", sampa_keys) # Has 43 keys


# Step 1: Make a sampa_ipa dict. How to: Go to SAMPA -> IPA tabelle, for every SAMPA char, check whether it 
# is in the CELEX korpus. If yes, look up example word from wiki in korpus and check whether it is the right sampa
# sign. If yes, look up on wiktionary example word in IPA and check whether output sign is correct.

sampa_ipa = dict()

# Vowels
sampa_ipa['i'] = 'i'
sampa_ipa[':'] = 'ː'
sampa_ipa['I'] = 'ɪ'
sampa_ipa['e'] = 'e'
sampa_ipa['E'] = 'ɛ'
sampa_ipa['y'] = 'y'   # meaning a real ü like in kühl
sampa_ipa['@'] = 'ə'
sampa_ipa['a'] = 'a'
sampa_ipa['u'] = 'u'
sampa_ipa['U'] = 'ʊ'
sampa_ipa['o'] = 'o'
sampa_ipa['O'] = 'ɔ'


# consonants
sampa_ipa['p'] = 'p'
sampa_ipa['b'] = 'b'
sampa_ipa['t'] = 't'
sampa_ipa['d'] = 'd'
sampa_ipa['k'] = 'k'
sampa_ipa['g'] = 'g'
sampa_ipa['f'] = 'f'
sampa_ipa['v'] = 'v'
sampa_ipa['s'] = 's'
sampa_ipa['z'] = 'z'
sampa_ipa['S'] = 'ʃ'
sampa_ipa['x'] = 'x'
sampa_ipa['h'] = 'h'
sampa_ipa['m'] = 'm'
sampa_ipa['n'] = 'n'
sampa_ipa['N'] = 'ŋ'
sampa_ipa['l'] = 'l'
sampa_ipa['r'] = 'r'
sampa_ipa['j'] = 'j'
sampa_ipa['Z'] = 'ʒ'
sampa_ipa['+'] = ''  # meaning a bit unclear
sampa_ipa['#'] = 'ˈ' # following syllabus carries primary intonation
sampa_ipa['|'] = 'ø' # meaning a bit unclear
sampa_ipa['/'] = 'œ' # usually SAMPA uses 9 instead of / for this 
sampa_ipa['Y'] = 'ʏ' # meaning more a 'oü' like in Müll

# These are 37 keys only, so 6 are missing. Remaining ones are:

# <GO>          not needed for alt. writing creation
# <PAD>         not needed for alt. writing creation
#    (SPACE)    not needed
# {             excluded some foreign words
# A             excluded some foreign words
# ~             excluded some foreign words


Length of phonetic dict is  40  and the keys are: 
 ['y', 's', 'e', '#', '@', 'i', 'k', ' ', 'm', 'U', 'I', 'r', 'p', 'S', 'u', 'o', 'h', '|', 'd', ':', '/', 'a', 'E', 'n', 'g', 'O', 'z', 'f', 'Z', 't', 'b', 'N', 'v', 'l', 'Y', 'x', 'j', '+', '<GO>', '<PAD>']


# Now convert the SAMPA words into IPA words:


In [9]:
ipa = []
for samp in phons:
    s = []
    for char in samp:
        s.append(sampa_ipa[char])
    ipa.append(''.join(s))
print("Amount of IPA samples is", len(words), ". Some samples are: \n")
print(" WORD         ===>        SAMPA        ===>        IPA")
for k in range(25100, 25110):
    print(words[k]," => ", phons[k]," => ", ipa[k])

Amount of IPA samples is 31675 . Some samples are: 

 WORD         ===>        SAMPA        ===>        IPA
toppsegel  =>  tOp#ze:g@l  =>  tɔpˈzeːgəl
toppsgast  =>  tOp+s#gast  =>  tɔpsˈgast
tor  =>  to:r  =>  toːr
tor  =>  to:r  =>  toːr
torbogen  =>  to:r#bi:g  =>  toːrˈbiːg
torf  =>  tOrf  =>  tɔrf
torferde  =>  tOrf#e:rd@  =>  tɔrfˈeːrdə
torfgewinnung  =>  tOrf#g@vIn+UN  =>  tɔrfˈgəvɪnʊŋ
torfig  =>  tOrf+Ix  =>  tɔrfɪx
torfmoor  =>  tOrf#mo:r  =>  tɔrfˈmoːr


## Now define the second dictionary, mapping IPA signs to graphemes (according to [here](https://de.wiktionary.org/wiki/Verzeichnis:Deutsch/Phoneme_und_Grapheme) ) 

Go to every phon and write down all graphemes (copying table, excluding very weird graphemes)

In [10]:
print(sampa_ipa.values())

dict_values(['i', 'ː', 'ɪ', 'e', 'ɛ', 'y', 'ə', 'a', 'u', 'ʊ', 'o', 'ɔ', 'p', 'b', 't', 'd', 'k', 'g', 'f', 'v', 's', 'z', 'ʃ', 'x', 'h', 'm', 'n', 'ŋ', 'l', 'r', 'j', 'ʒ', '', 'ˈ', 'ø', 'œ', 'ʏ'])


In [11]:
ipa_graph = dict()
ipa_graph['t'] = ['t', 'd', 'tt'] # excluded dt (mostly in Stadt) and th (Methode) -> hard but not do more than 3 opts.
ipa_graph['ə'] = ['e']
ipa_graph['n'] = ['n', 'nn']
ipa_graph['s'] = ['s', 'ss'] # excluded t for Patience, ce for Farce, zz for Jazz, ß (no occ.), c for Sauce, z for Quiz
ipa_graph['a'] = ['a', 'ah']
ipa_graph['r'] = ['r', 'rr'] # excluded rrh for Zirrhose/Myrrhe, rh for Rhythmus
ipa_graph['l'] = ['l', 'll']
ipa_graph['ɛ'] = ['e', 'ae']
ipa_graph['f'] = ['f', 'v', 'ff'] # excluded ph for Physik
ipa_graph['g'] = ['g', 'gh'] # excluded gg for Bagger
ipa_graph['ɪ'] = ['i']
ipa_graph['k'] = ['k', 'ck', 'c'] # Excluded cch for Zucchini, gg for Flaggschiff, qu for Boutique, kk for Akkordeon
                  #  qu chars are usually kv ipa (tracked below), g for Krieg, ch for Chor
ipa_graph['m'] = ['m', 'mm']
ipa_graph['b'] = ['b', 'bb']
ipa_graph['ʃ'] = ['sch', 's'] # excluded sk for Ski, sh for Sheriff, Show and ch for Recherche 
ipa_graph['d'] = ['d','dd']
ipa_graph['p'] = ['p', 'b', 'pp'] # excluded bb for abebben or schrubben
ipa_graph['ŋ'] = ['ng','n']
ipa_graph['ɔ'] = ['o'] # excluded ch for Chauffeur (very rare exception)
ipa_graph['v'] = ['w', 'v']
ipa_graph['ʊ'] = ['u']
ipa_graph['z'] = ['s'] # excluded zz for Blizzard, Puzzle and z for zoomen, bulldozer (since only in foreign words)
ipa_graph['h'] = ['h']
ipa_graph['i'] = ['i'] # excluding y (Baby/Party/Hockey) only 10 words in corpus
ipa_graph['ʏ'] = ['ue', 'u'] # The corpus is weird here and writes Druck as drʏk, i.e. "Drück" rather than drʊk
ipa_graph['x'] = ['ch']
ipa_graph['e'] = ['e'] # excluded ee for Kaffee since IPA would be eː
ipa_graph['j'] = ['j', 'y']
ipa_graph['u'] = ['u'] # excluded ou like in Boutique
ipa_graph['o'] = ['o'] # not needed anyways since o always followed by ː
ipa_graph['œ'] = ['oe']
ipa_graph['y'] = ['y']
ipa_graph['ʒ'] = ['g', 'j'] # no wiki entry, self generated. For Garage or Jury


# 2 character keys:
ipa_graph['ts'] = ['z', 'ts', 'tz'] # excluded zz for Pizza/Skizze, c for circa, Penicillin, tts for trittst
            # t for Aktion, Negation, Infektion, Proportion, ...
ipa_graph['aː'] = ['a', 'ah', 'aa']
ipa_graph['ai'] = ['ei', 'ai'] # excluded ail for Detail, aill for Medaillon, aille for Medaille and y for Nylon
ipa_graph['iː'] = ['ie', 'i', 'ih'] # excluded ieh for Entziehung
ipa_graph['eː'] = ['e', 'ee', 'eh'] # excluded et like in Bidet
ipa_graph['ɛː'] = ['ae', 'aeh']
ipa_graph['uː'] = ['u', 'uh'] # excluded ou like in Ragout, Limousine and oo like on zoomen/Cartoon
ipa_graph['oː'] = ['o', 'oh', 'oo'] # excluded au for aubergine/sauce and eau for plateau, Niveau
ipa_graph['yː'] = ['ue', 'ueh', 'y'] # excluded uet like in Debüt and u like in deja-vu
ipa_graph['ɔy'] = ['eu', 'aeu'] # instead of what wiki calls ɔɪ̯, excluded oi for Boiler and oy for Boykott
ipa_graph['ks'] = ['chs', 'x', 'ks'] #excluded gs like in legst/bugsieren, ggs like in eggst (?), cks like in zwecks, gs (legst)
ipa_graph['øː'] = ['oe', 'oeh'] # excluded eu like in Ingenieur and eue like in Queue (?)
ipa_graph['kv'] = ['qu']


print(len(sampa_ipa), len(ipa_graph))
# We had 36 keys in sampa_ipa dict, now we have 46 already in ipa_graph due to 2-phoneme-groups
# But still there are one-char-values in sampa_ipa which are not keys in ipa_graph. Let us print them:
for key in sampa_ipa.values():
    if key not in ipa_graph.keys():
        print(key)
        
# Okay 4 are missing:
ipa_graph['ː'] = [''] # not needed anyways since ː always occurs after vowel
ipa_graph['ˈ'] = [''] # Just a pronounciation symbol, does not carry meaning for spelling
# Then the empty string '' is not needed as key
# Then ø only occurs followed by a ː


37 46
ː

ˈ
ø


#### Pretty tough to draw the boundary between which phons rarely but "regularly" translate to grapheme strings ([k] to kk like in Akkordeon, Mokka is still regular?) whereas [ʃ] (sch) to ch like in Champagner, Recherche or Lunch is irregular?

## Use the IPA->Graphem dict to create alternative writings

In [12]:
def split_word(word, ipa_graph):
    """
    Splits up an IPA word into a list of lists each with the possible replacement grapheme for each phoneme
    
    Parameters:
    ----------
    WORD       {list} in IPA notation
    IPA_GRAPH  {dict} mapping IPA symbols to possible grapheme sequences
    
    Returns:
    ---------
    CHARS      {list} containing lists with possible grapheme sequences
    """
    
    
    chars = []
    single_key = True
    for ind in range(len(word)-1):
        
        if single_key: 
            if word[ind:ind+2] in ipa_graph:
                chars.append(ipa_graph[word[ind:ind+2]])
                single_key = False
            else:
                chars.append(ipa_graph[word[ind]])
        else:
            single_key = True
            
    chars.append(ipa_graph[word[ind+1]])
    
    return chars
    


# The cell below generates the alternative writings (takes some time...)

In [ ]:
a = np.array([1,0,0,1,0])
d = dict()
d[str(a)] = 12

e = np.random.random((2,2,2,2,2))
f = [np.asscalar(aa) for aa in a]
print(f)
d[tuple(a)] = 121
print(d)


In [ ]:
%%timeit
d[np.str(a)]

In [ ]:
%%timeit
e[a[0],a[1],a[2],a[3], a[4]]

In [ ]:
%%timeit
d[tuple(a)]

In [ ]:
all_writings = []
m = 0

for ind,ip in enumerate(ipa):
    if ind % 200 == 0:
        print("Currently examining word ", ind)
        
    word_lists = split_word(ip, ipa_graph)
    alt_write_raw = list(itertools.product(*word_lists))
    alt_write = [''.join(a) for a in alt_write_raw]
    try:
        alt_write.remove(words[ind])
    except ValueError:
        _ = 1
        
    all_writings.append(alt_write)
    
    if len(alt_write) > m:
        print(m,ind)
        m = len(alt_write)
        
        
print("DONE! Alternative writings generated for ", ind, "words. Resulting list has", len(all_writings), 'entries.')
np.save('celex_all_writings', all_writings)

# Extract childlex data from downloaded CSV and clean it up

In [13]:
import csv
path = 'data/childlex_6-8_lemmata.csv'
forbidden = "_,.!?*:=&%- /\\–•»«…›‹()[]{}’'"


def rem_um(word):
    """
    Converts a word with German umlauts (ü,ä,ö) into a word without
    """
        
    umlautfree = str()
    for char in word:
        if char == 'ä':
            umlautfree += 'ae'
        elif char == 'ü':
            umlautfree += 'ue'
        elif char == 'ö':
            umlautfree += 'oe'
        elif char == 'ß':
            umlautfree += 'ss'
        else:
            umlautfree += char
    return umlautfree


with open(path, 'r') as csvfile:
    
    raw_file = csv.reader(csvfile,delimiter=';')
    raw_data = []
    
    
    for line in raw_file:
        umlautfree = rem_um(line[0].lower())
        
       # data cleanup
        has_digit = any(char.isdigit() for char in umlautfree)
        has_sonderz = any(char in forbidden for char in umlautfree)
        
        if not has_digit and not has_sonderz:
        
            raw_data.append(umlautfree)
        
        
    
childlex_words_all = raw_data
print("Amount of words in childlex", len(childlex_words_all))
    
chars = []
for word in childlex_words_all:
    for char in word:
        if char not in chars:
            chars.append(char)
    
print("Set of characters in childlex words is", chars)

Amount of words in childlex 9769
Set of characters in childlex words is ['a', 'e', 'c', 'h', 'z', 'n', 'd', 'f', 'm', 'l', 'i', 'r', 'g', 's', 't', 'u', 'o', 'b', 'k', 'p', 'q', 'w', 'v', 'x', 'j', 'y']


#### Look up phonetic sequences in CELEX Database. 
#### Problem: We do not have phonetic sequences yet. Idea: Retrieve as many phonetic sequences as possible from the CELEX database, convert this database into numerical values and save it.

In [14]:
childlex_phons = []
childlex_words = []
for word in childlex_words_all:
    if word in words:
        ind = words.index(word)
        childlex_phons.append(phons[ind])
        childlex_words.append(words[ind])

In [15]:
a =  childlex_phons.count('')
print("From the", len(childlex_words_all),"words in CHILDLEX database", len(childlex_words),"were succesfully retrieved")

From the 9769 words in CHILDLEX database 5885 were succesfully retrieved


In [16]:
# Save the childlex dataset
((phons_num, words_num), (phon_dict, word_dict)) = str_to_num_dataset(childlex_phons,childlex_words)
print(len(word_dict),word_dict)
print(len(phon_dict),phon_dict)
print(words_num.shape, phons_num.shape)
print(words_num[321,:], phons_num[321,:])
np.savez("data/childlex.npz", words=words_num, phons=phons_num, word_dict=word_dict, phon_dict=phon_dict)

29 {'y': 1, 's': 2, 'e': 3, 'i': 4, 'w': 5, 'k': 6, ' ': 7, 'm': 8, 'r': 9, 'q': 10, 'p': 11, 'c': 12, 'o': 13, 'u': 14, 'h': 15, 'd': 16, 'a': 17, 'n': 18, 'g': 19, 'z': 20, 'f': 21, 't': 22, 'b': 23, 'v': 24, 'l': 25, 'x': 26, 'j': 27, '<GO>': 28, '<PAD>': 29}
40 {'y': 1, 's': 2, 'e': 3, '#': 4, '@': 5, 'i': 6, 'k': 7, ' ': 8, 'm': 9, 'U': 10, 'I': 11, 'r': 12, 'p': 13, 'S': 14, 'h': 15, 'u': 16, '|': 17, 'o': 18, 'd': 19, ':': 20, '/': 21, 'a': 22, 'E': 23, 'n': 24, 'g': 25, 'O': 26, 'z': 27, 'f': 28, 'Z': 29, 't': 30, 'b': 31, 'N': 32, 'v': 33, 'l': 34, 'Y': 35, 'x': 36, 'j': 37, '+': 38, '<GO>': 39, '<PAD>': 40}
(5885, 20) (5885, 15)
[28 29 29 29 29 29 29 29 29 29 29 29 29 29 17 14  2  2  3 18] [39 40 40 40 40 40 40 40 40 22 16  2 38  5 24]


### Extract data from the FIBEL dataset
##### Focus on the "Mia and Mo" Fibel

In [17]:
fibel_path = "/Users/jannisborn/Dropbox/GitHub/LSTM/LdS_bLSTM/Code/data/Fibelwörter.txt"


with open(fibel_path, 'r') as txtfile:
    fibel_words = []
    for line in txtfile.read().split(','):
        # Use lowercase letters only, remove leading whitespacetake care of line breaks.
        if '\n' in line:    
            for item in line.split('\n'):
                if item.lstrip().lower() not in fibel_words:
                    fibel_words.append(rem_um(item.lstrip().lower()))
        elif line.lstrip().lower() not in fibel_words:
            fibel_words.append(rem_um(line.lstrip().lower()))
        
lektions_inds = [9,14,20,28,36,46,58,77,99,121,154,174]

fibel_phons = []
k=0
for w in fibel_words:
    if w in words:
        ind = words.index(w)
        fibel_phons.append(phons[ind])
    else:
        fibel_phons.append('NO')
        k +=1
print("The dataset has "+str(len(fibel_words))+" from which "+str(len(fibel_words)-k)+" could be retrieved from celex")

The dataset has 175 from which 115 could be retrieved from celex


#### Generate the SAMPA spellings for the missing words. First step: Copy the IPA values from wiktionary + convert IPA into SAMPA 

In [29]:
ipa_sampa = dict(zip(sampa_ipa.values(), sampa_ipa.keys()))

fibel_phons[fibel_words.index('mia')] = ''.join([ipa_sampa[w] for w in 'miːa'])
fibel_phons[fibel_words.index('mo')] = ''.join([ipa_sampa[w] for w in 'mo'])
fibel_phons[fibel_words.index('mimi')] = ''.join([ipa_sampa[w] for w in 'mimi'])
fibel_phons[fibel_words.index('im')] = ''.join([ipa_sampa[w] for w in 'ɪm'])
fibel_phons[fibel_words.index('am')] = ''.join([ipa_sampa[w] for w in 'am'])
fibel_phons[fibel_words.index('momo')] = ''.join([ipa_sampa[w] for w in 'momo'])
fibel_phons[fibel_words.index('omi')] = ''.join([ipa_sampa[w] for w in 'oːmiː'])
fibel_phons[fibel_words.index('radio')] = ''.join([ipa_sampa[w] for w in 'reɪdɪəʊ'])
fibel_phons[fibel_words.index('sissi')] = ''.join([ipa_sampa[w] for w in 'zɪsɪː'])
fibel_phons[fibel_words.index('susi')] = ''.join([ipa_sampa[w] for w in 'susi'])
fibel_phons[fibel_words.index('oli')] = ''.join([ipa_sampa[w] for w in 'ɔliː'])
fibel_phons[fibel_words.index('salami')] = ''.join([ipa_sampa[w] for w in 'zaˈlaːmi'])
fibel_phons[fibel_words.index('ist')] = ''.join([ipa_sampa[w] for w in 'ɪst'])
fibel_phons[fibel_words.index('tim')] = ''.join([ipa_sampa[w] for w in 'tɪm'])
fibel_phons[fibel_words.index('tom')] = ''.join([ipa_sampa[w] for w in 'tɔm'])
fibel_phons[fibel_words.index('mario')] = ''.join([ipa_sampa[w] for w in 'ˈmaːrio'])
fibel_phons[fibel_words.index('isa')] = ''.join([ipa_sampa[w] for w in 'iːza'])
fibel_phons[fibel_words.index('maria')] = ''.join([ipa_sampa[w] for w in 'maˈriːa'])
fibel_phons[fibel_words.index('rosarot')] = ''.join([ipa_sampa[w] for w in 'ˈroːzaˈroːt'])
fibel_phons[fibel_words.index('nimm')] = ''.join([ipa_sampa[w] for w in 'nɪm'])
fibel_phons[fibel_words.index('nimmt')] = ''.join([ipa_sampa[w] for w in 'nɪmt'])
fibel_phons[fibel_words.index('nina')] = ''.join([ipa_sampa[w] for w in 'niːna'])
fibel_phons[fibel_words.index('rosinen')] = ''.join([ipa_sampa[w] for w in 'roˈziːnən'])
fibel_phons[fibel_words.index('anna')] = ''.join([ipa_sampa[w] for w in 'ˈana'])
fibel_phons[fibel_words.index('bananen')] = ''.join([ipa_sampa[w] for w in 'baˈnaːnən'])
fibel_phons[fibel_words.index('birnen')] = ''.join([ipa_sampa[w] for w in 'bɪrnən'])
fibel_phons[fibel_words.index('nuesse')] = ''.join([ipa_sampa[w] for w in 'nʏsə'])
fibel_phons[fibel_words.index('orangen')] = ''.join([ipa_sampa[w] for w in 'oˈraŋʒn'])
fibel_phons[fibel_words.index('weintrauben')] = ''.join([ipa_sampa[w] for w in 'ˈvaintraʊbn'])
fibel_phons[fibel_words.index('maroni')] = ''.join([ipa_sampa[w] for w in 'maˈroːni'])
fibel_phons[fibel_words.index('erna')] = ''.join([ipa_sampa[w] for w in 'ˈɛrna'])
fibel_phons[fibel_words.index('die')] = ''.join([ipa_sampa[w] for w in 'diː'])
fibel_phons[fibel_words.index('das')] = ''.join([ipa_sampa[w] for w in 'das'])
fibel_phons[fibel_words.index('sind')] = ''.join([ipa_sampa[w] for w in 'zɪnt'])
fibel_phons[fibel_words.index('domino')] = ''.join([ipa_sampa[w] for w in 'ˈdoːmino'])
fibel_phons[fibel_words.index('indianer')] = ''.join([ipa_sampa[w] for w in 'ɪnˈdiaːnər'])
fibel_phons[fibel_words.index('tonio')] = ''.join([ipa_sampa[w] for w in 'tɔnioː'])
fibel_phons[fibel_words.index('sagt')] = ''.join([ipa_sampa[w] for w in 'zaːkt'])
fibel_phons[fibel_words.index('italien')] = ''.join([ipa_sampa[w] for w in 'iˈtaːliən'])
fibel_phons[fibel_words.index('ute')] = ''.join([ipa_sampa[w] for w in 'uːtə'])
fibel_phons[fibel_words.index('umarmt')] = ''.join([ipa_sampa[w] for w in 'ʊmˈarmt'])
fibel_phons[fibel_words.index('runter')] = ''.join([ipa_sampa[w] for w in 'rʊntər'])
fibel_phons[fibel_words.index('turnen')] = ''.join([ipa_sampa[w] for w in 'tʊrnən'])
fibel_phons[fibel_words.index('dem')] = ''.join([ipa_sampa[w] for w in 'deːm'])
fibel_phons[fibel_words.index('rennt')] = ''.join([ipa_sampa[w] for w in 'rɛnt'])
fibel_phons[fibel_words.index('ene')] = ''.join([ipa_sampa[w] for w in 'eːnə'])
fibel_phons[fibel_words.index('mene')] = ''.join([ipa_sampa[w] for w in 'meːnə'])
fibel_phons[fibel_words.index('mu')] = ''.join([ipa_sampa[w] for w in 'muː'])
fibel_phons[fibel_words.index('turnt')] = ''.join([ipa_sampa[w] for w in 'ˈtʊrnt'])
fibel_phons[fibel_words.index('miauen')] = ''.join([ipa_sampa[w] for w in 'miˈaʊən'])
fibel_phons[fibel_words.index('raus')] = ''.join([ipa_sampa[w] for w in 'raʊs'])
fibel_phons[fibel_words.index('mauert')] = ''.join([ipa_sampa[w] for w in 'tɔm'])
fibel_phons[fibel_words.index('saust')] = ''.join([ipa_sampa[w] for w in 'zaʊst'])
fibel_phons[fibel_words.index('rosaroten')] = ''.join([ipa_sampa[w] for w in 'roːzaˈroːtn'])
fibel_phons[fibel_words.index('martin')] = ''.join([ipa_sampa[w] for w in 'ˈmartiːn'])
fibel_phons[fibel_words.index('martins')] = ''.join([ipa_sampa[w] for w in 'ˈmartiːns'])
fibel_phons[fibel_words.index('eine')] = ''.join([ipa_sampa[w] for w in 'ˈaɪnə'])
fibel_phons[fibel_words.index('miaut')] = ''.join([ipa_sampa[w] for w in 'miˈaʊt'])
fibel_phons[fibel_words.index('otto')] = ''.join([ipa_sampa[w] for w in 'ˈɔto'])
fibel_phons[fibel_words.index('isst')] = ''.join([ipa_sampa[w] for w in 'ɪst'])

try:
    fibel_phons.index('NO')
except ValueError:
    print("YAY, all words have phonetic transcript")

YAY, all words have phonetic transcript


### Save the Fibel dataset

In [30]:
((phons_num, words_num), (phon_dict, word_dict)) = str_to_num_dataset(childlex_phons,childlex_words)
np.savez("data/childlex.npz", words=words_num, phons=phons_num, word_dict=word_dict, phon_dict=phon_dict)

In [ ]:
# Insight 1: If you have k different classes, then k is the third 
#    dimension of the logits and then the target matrix must not 
#    contain values higher than k-1 (i.e. labels are 0, ..., k)

# Insight 2: sequence_loss expects unnormalized logits (BEFORE softmax!)
# -> This makes testing for me harder, since I cannot simply use values
# with a sum over 1 (will get normalized)